# Inference Testing

## Model Checking

Run a sample MNIST image through the saved model and observe the outputs.
Format the predicted outputs to make it consumable.

## Load Saved Model

In [82]:
from pathlib import Path

import torch

In [83]:
def load_model(jit_model_path):
    model = None
    try:
        model = torch.jit.load(jit_model_path)
    except RuntimeError:
        print(f"Unrecognized model format. Expecting model in TorchScript format. "
              f"{jit_model_path}")

    return model

In [84]:
MODEL_ARTIFACT = Path("./artifacts/SimpleCNN_mnist_jit.pt")
model = load_model(MODEL_ARTIFACT)
print(model)

RecursiveScriptModule(
  original_name=SimpleCNN
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (dropout1): RecursiveScriptModule(original_name=Dropout)
  (dropout2): RecursiveScriptModule(original_name=Dropout)
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
)


## Load Image

In [95]:
import os
import random

from PIL import Image

def get_sample_image():
    SAMPLE_IMAGES_DIR = Path("dataset/MNIST/sample_images")
    sample_images = os.listdir(SAMPLE_IMAGES_DIR)
    sample_image = SAMPLE_IMAGES_DIR / random.choice(sample_images)

    return sample_image

def show_image_stats(image_path):
    img = Image.open(image_path)
    print(f"Image: {image_path.name}", end="  ")
    print(f"Format: {img.format}  Size: {img.size}  Mode: {img.mode}")
    img.show()


## Transform Image

In [87]:
from torchvision.transforms import ToTensor, Normalize

def tensorify_image(image_path):
    img = Image.open(image_path)
    tensor_img = ToTensor()(img)
    norm_img = Normalize((0.1307,), (0.3081,))(tensor_img)
    return norm_img.unsqueeze(0)

In [93]:
sample_image.as_posix()[-5]

'1'

In [96]:
for _ in range(10):
    sample_image = get_sample_image()
    show_image_stats(sample_image)
    img_tensor = tensorify_image(sample_image)
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
    pred = output.argmax(dim=1, keepdim=True).item()
    print(f"Actual {sample_image.as_posix()[-5]}  Predicted: {pred}")

Image: image_43214_1.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 1  Predicted: 7
Image: image_31386_1.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 1  Predicted: 7
Image: image_45146_0.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 0  Predicted: 7
Image: image_10590_7.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 7  Predicted: 7
Image: image_8610_1.png  Format: PNG  Size: (28, 28)  Mode: L


...

Actual 4  Predicted: 7
Image: image_34571_4.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 4  Predicted: 7
Image: image_36187_2.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 2  Predicted: 7
Image: image_9470_7.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 7  Predicted: 7
Image: image_1394_1.png  Format: PNG  Size: (28, 28)  Mode: L


Actual 1  Predicted: 7


In [97]:
isinstance(img_tensor, torch.Tensor)

True

## MNIST Dataset

```
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])

mnist_train = datasets.MNIST("./dataset", train=True, transform=transform)
mnist_test = datasets.MNIST("./dataset", train=False, transform=transform)

mnist_train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64)
mnist_test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100)
```

```
for data, target in mnist_train_loader:
    break
data.shape
```